In [1]:
import sys,os
import numpy as np
import trimesh
import pyrender
import matplotlib.pyplot as plt
from sklearn.neighbors import KDTree
import open3d as o3d
from utils.tools import *
import scipy
import scipy.sparse.linalg
import scipy.signal

d:\software\anaconda\lib\site-packages\pyglet\libs\win32\__init__.py:326: UserWarning: Could not set COM MTA mode. Unexpected behavior may occur.
  warnings.warn("Could not set COM MTA mode. Unexpected behavior may occur.")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


1. Uniform Laplace

Mean Curvature and Gaussian Curvature

In [2]:
curve_path = '../data/meshes/curvatures/'
mesh_path = '../data/meshes/'

path_plane = curve_path + 'plane.obj'
path_lilium = curve_path + 'lilium_s.obj'
path_bumpy_cube = mesh_path + 'bumpy-cube.obj'

In [3]:
for path in [path_plane, path_lilium, path_bumpy_cube]:
    print('Current path:', path)    
    tm = trimesh.load_mesh(path) 
    dc = DiscreteCurvature(tm)

    H = dc.compute_curvature('mean')
    K = dc.compute_curvature('gaussian')
    
    print('\n')
    dc.show(H)
    dc.show(K)

Current path: ../data/meshes/curvatures/plane.obj


The range of curvature: 0.000, 0.000
The range of curvature: 0.000, 0.000
Current path: ../data/meshes/curvatures/lilium_s.obj


The range of curvature: 0.000, 0.036
The range of curvature: -8.997, 8.564
Current path: ../data/meshes/bumpy-cube.obj


The range of curvature: 0.000, 0.005
The range of curvature: -1.346, 5.273


3. Non-uniform (Discrete Laplace-Beltrami)

In [4]:
for path in [path_plane, path_lilium, path_bumpy_cube]:   
    print('Current path:', path)     
    tm = trimesh.load_mesh(path) 
    dc = DiscreteCurvature(tm)

    H_cot = dc.compute_curvature('non-uniform')
    dc.show(H_cot)
    print('\n')

Current path: ../data/meshes/curvatures/plane.obj
The range of curvature: 0.000, 0.000


Current path: ../data/meshes/curvatures/lilium_s.obj
The range of curvature: 0.009, 4.485


Current path: ../data/meshes/bumpy-cube.obj
The range of curvature: 0.001, 2.546




4. Modal analysis (Decomposition)

In [5]:
path_decomp = '../data/meshes/decomposition'
path_armadillo = path_decomp + '/armadillo.obj'
path_dec_cow = path_decomp + '/cow_small_manifold2.obj'

tm = trimesh.load_mesh(path_armadillo) 
dc = DiscreteCurvature(tm)

In [11]:
k_list = [5, 15, 35, 65, 500, 1000]
for path in [path_armadillo]:
    tm = trimesh.load_mesh(path) 
    dc = DiscreteCurvature(tm)
    
    for k in k_list:
        print('k:', k)
        reconstruct_tm = dc.reconstruct(k)
        reconstruct_tm.export('../results/armadillo'+str(k)+'.obj')
        
        recon_mesh = dc.tm2o3d(reconstruct_tm)
        o3d.visualization.draw_geometries([recon_mesh])

k: 5
k: 15
k: 35
k: 65
k: 500
k: 1000


Laplacian Mesh Smoothing

5. Explicit Laplacian Mesh Smoothing

In [14]:
path_plane = '../data/meshes/smoothing/plane_ns.obj'
path_fandisk = '../data/meshes/smoothing/fandisk_ns.obj'

tm_plane = trimesh.load_mesh(path_plane)
tm_fandisk = trimesh.load_mesh(path_fandisk)


smooth_plane = DiscreteCurvature(tm_plane)
smooth_fandisk = DiscreteCurvature(tm_fandisk)

In [15]:
plane_mesh = smooth_plane.tm2o3d(tm_plane)
fandisk_mesh = smooth_fandisk.tm2o3d(tm_fandisk)
o3d.visualization.draw_geometries([plane_mesh])
o3d.visualization.draw_geometries([fandisk_mesh])

In [39]:
plane_lam_list = [1e-7, 5e-7, 1e-6]
for lam in plane_lam_list:
    print(f'lambda: {lam}')
    new_tm = smooth_plane.smooth(50, lam)

    new_mesh = smooth_plane.tm2o3d(new_tm)
    o3d.visualization.draw_geometries([new_mesh])

lambda: 0.001


In [41]:
fandisk_lam_list = [1e-5, 5e-5, 1e-4]
for lam in fandisk_lam_list:
    print(f'lambda: {lam}')
    new_tm = smooth_fandisk.smooth(5, lam)

    new_mesh = smooth_fandisk.tm2o3d(new_tm)
    o3d.visualization.draw_geometries([new_mesh])

lambda: 0.001


6. Implicit Laplacian Mesh Smoothing

In [ ]:
plane_lam_list = [1e-7, 5e-7, 1e-6, 1e-5]
iterations = [50, 100, 150]
# Smoothing the plane object with different step size and iterations
for lam in plane_lam_list:
    for iter in iterations:
        print(f'lambda: {lam} \t iter: {iter}')
        imp_tm = smooth_plane.smooth(iter, lam, 'implicit')

        # convert result to o3d mesh
        imp_mesh = smooth_plane.tm2o3d(imp_tm)
        
        print('Show implicit smoothing result')
        o3d.visualization.draw_geometries([imp_mesh])

In [44]:
fandisk_lam_list = [1e-5, 5e-5, 1e-4, 1e-3]
iterations = [50, 100, 150]
# Smoothing the fandisk object with different step size and iterations
for lam in fandisk_lam_list:
    for iter in iterations:
        print(f'lambda: {lam} \t iter: {iter}')
        imp_tm = smooth_fandisk.smooth(iter, lam, 'implicit')

        # convert result to o3d mesh
        imp_mesh = smooth_fandisk.tm2o3d(imp_tm)
        
        print('Show implicit smoothing result')
        o3d.visualization.draw_geometries([imp_mesh])

lambda: 0.01 	 iter: 50
Show explict smoothing result
lambda: 0.01 	 iter: 100
Show explict smoothing result
lambda: 0.01 	 iter: 150
Show explict smoothing result


7. Denosing

In [45]:
path_bunny = '../data/meshes/bunny.obj'

In [70]:
noise_scales = [0.005, 0.01, 0.05]
for noise in noise_scales:
    bunny_mesh = trimesh.load_mesh(path_bunny)
    
    denoise_bunny = DiscreteCurvature(bunny_mesh)
    denoise_bunny.generate_noise(noise)

    # Show the noisy object
    mesh = denoise_bunny.tm2o3d(bunny_mesh)
    o3d.visualization.draw_geometries([mesh])

    # Smooth the object
    # TODO Users can change the parameters to try different step size and iterations
    smooth_tm = denoise_bunny.smooth(100, 1e-5, 'implicit')
    imp_mesh = denoise_bunny.tm2o3d(smooth_tm)
    o3d.visualization.draw_geometries([imp_mesh])